In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('cvrm_master.csv')
from cleaning_function import *

/var/folders/26/l89mqgp17zj82yvlmd3z3r200000ks/T/ipykernel_32578/1321871251.py:3: DtypeWarning: Columns (1697,1698) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('cvrm_master.csv')


In [2]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data = drop_null_columns(data, 0.6)

In [3]:
numerical_cols = data.select_dtypes(include=['number']).columns
categorical_cols = data.select_dtypes(exclude=['number']).columns
df_mean = data.copy()
df_mean[numerical_cols] = df_mean[numerical_cols].fillna(df_mean[numerical_cols].mean())

df_median = data.copy()
df_median[numerical_cols] = df_median[numerical_cols].fillna(df_median[numerical_cols].median())

df_mean.to_csv('mean_imputation.csv',index = False)
df_median.to_csv('median_imputation.csv', index = False)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings("ignore")

data = drop_null_columns(data, 0.6)
data.replace([np.inf, -np.inf], np.nan, inplace=True)
numerical_cols = data.select_dtypes(include=['number']).columns
categorical_cols = data.select_dtypes(exclude=['number']).columns

df_mean = data.copy()
df_mean[numerical_cols] = df_mean[numerical_cols].fillna(df_mean[numerical_cols].mean())
df_median = data.copy()
df_median[numerical_cols] = df_median[numerical_cols].fillna(df_median[numerical_cols].median())

df_mean.to_csv('mean_imputation.csv', index=False)
df_median.to_csv('median_imputation.csv', index=False)
kf = KFold(n_splits=5, shuffle=True, random_state=42)


def compute_rmse(df, imputation_type):
    all_rmse = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(df[numerical_cols])):
        print(f"Starting {imputation_type} Imputation - Fold {fold + 1}")
        train_data, test_data = df.iloc[train_idx], df.iloc[test_idx]
        scaler = StandardScaler()
        train_scaled = scaler.fit_transform(train_data[numerical_cols])
        test_scaled = scaler.transform(test_data[numerical_cols])
        pca = PCA(n_components=min(20, train_scaled.shape[1])) 
        train_reduced = pca.fit_transform(train_scaled)
        test_reduced = pca.transform(test_scaled)

        model = Ridge(alpha=1.0)
        model.fit(train_reduced, train_scaled) 
        test_predictions = model.predict(test_reduced)
        mse = mean_squared_error(test_scaled, test_predictions)
        rmse = np.sqrt(mse)
        all_rmse.append(rmse)

        print(f"{imputation_type} - Fold {fold + 1} RMSE: {rmse}")

    avg_rmse = sum(all_rmse) / len(all_rmse)
    print(f"Average RMSE for {imputation_type} Imputation: {avg_rmse}")
    return avg_rmse

mean_rmse = compute_rmse(df_mean, "Mean")
median_rmse = compute_rmse(df_median, "Median")

print(f"Final RMSE Comparison:\nMean Imputation: {mean_rmse}\nMedian Imputation: {median_rmse}")


Starting Mean Imputation - Fold 1
Mean - Fold 1 RMSE: 0.6203075757012638
Starting Mean Imputation - Fold 2
Mean - Fold 2 RMSE: 0.6200968725832966
Starting Mean Imputation - Fold 3
Mean - Fold 3 RMSE: 19.041173206061067
Starting Mean Imputation - Fold 4
Mean - Fold 4 RMSE: 0.6162126549721304
Starting Mean Imputation - Fold 5
Mean - Fold 5 RMSE: 0.5763441819180056
Average RMSE for Mean Imputation: 4.294826898247154
Starting Median Imputation - Fold 1
Median - Fold 1 RMSE: 0.6194973315398064
Starting Median Imputation - Fold 2
Median - Fold 2 RMSE: 0.619297478267454
Starting Median Imputation - Fold 3
Median - Fold 3 RMSE: 0.7991300017604913
Starting Median Imputation - Fold 4
Median - Fold 4 RMSE: 0.6152047913944871
Starting Median Imputation - Fold 5
Median - Fold 5 RMSE: 0.5754361013402166
Average RMSE for Median Imputation: 0.645713140860491
Final RMSE Comparison:
Mean Imputation: 4.294826898247154
Median Imputation: 0.645713140860491
